Welcome back!

In this lab, we will observe how weights and bias are trained over epochs.

Let's begin with importing the libraries and read the dataset. We will also break the dataset into features and labels.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
#IGNORE ANY WARNINGS THAT MAY APPEAR ON THIS CELL

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('data/data.csv')

In [ ]:
x = data[['radius_mean', 'perimeter_mean']]
y = data[['diagnosis']]

We are using the same dataset as in the last lab- view the inputs and labels.

In [ ]:
x.head()

In [ ]:
y.head()

Using the same process as the last lab, we will normalize the inputs, and encode the labels.

In [ ]:
#Normalize Input
stats = x.describe().T
mu, sigma = stats['mean'], stats['std']

In [ ]:
x_norm = (x-mu)/sigma

In [ ]:
x_norm

In [ ]:
rule = lambda val: 1 if val=='M' else 0

In [ ]:
y=y['diagnosis'].apply(rule)

In [ ]:
y.tail()

Now we will proceed to splitting the data. No changes so far-

In [ ]:
HP_epochs = 200
HP_batch_size = 16
HP_lr = 0.001

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x_norm,y,random_state=42)

Our dataset is small. But for big data, or larger datasets, we can use TensorFlow's Dataset API. 
<br/>
We additionally shuffle the dataset to provide randomness. 

Because we are going to simulate the training process over epochs, we will also repeat our dataset. 

We will also divide our data into fixed size batches.

In [ ]:
celldata = tf.data.Dataset.from_tensor_slices((xtrain,ytrain))
celldata = celldata.shuffle(100).repeat(HP_epochs).batch(HP_batch_size)

This is the simplest way to create a dataset. 

dataset = tf.data.Dataset.from_tensor_slices(any_tensor_or_numpy_array_or_pandas_dataset)

Now we will place an iterator on top of our dataset to loop into the actual data.

In [ ]:
itr = celldata.__iter__()
#IGNORE ANY WARNINGS THAT MAY APPEAR ON THIS CELL

Now, we will initialize our weights and bias. We are initializing the weights with random numbers, normally distributed from the mean 0, with a standard deviation of 0.5. 


In [ ]:
# we have only 2 features and 1 label
# One weight for each feature
w = tf.random.normal( ( 2 , 1 ), mean = 0, stddev =0.5,dtype=tf.dtypes.double ) 
# One bias for every output 
# our only output is the label prediction on 1 layer
b = 0

Have a look at the initial weights

In [ ]:
print(w)

In [ ]:
# define an empty array to store all the errors
# errors will be generated from each training simulation
err = []

Let's define a dummy classifier to encode our predictions into 1 and 0

In [ ]:

def myclassifier(vals):
  res = []
  rule = lambda r: 1.0 if r>=0.5 else 0.0
  for val in vals:
    res.append(rule(val))
  return res


And let's begin the training simulation!

TensorFlow functions to observe


*   tensordot - returns dot product
*   cast - datatype casting on tensors
*   reduce_mean - mean of TensorFlow datasets



In [ ]:
itrs_per_epoch = int(len(xtrain)/HP_batch_size)
# Each iteration represents 1 epoch
for i in range(HP_epochs):
  per_iter_err = []
  for j in range(itrs_per_epoch):
    # get data and labels in fixed batch sizes
    data, labels = itr.get_next()
    
    # simplest representation of ML 
    # y = mx + c
    y = tf.tensordot(data, w, axes=1) + b
    # decode the results, and typecast to double
    y = myclassifier(y)
    y = tf.cast(y, tf.dtypes.double)
    # typecast the known labels to double
    labels = tf.cast(labels, tf.dtypes.double)
    # calculate the error
    avg_error = tf.reduce_mean(tf.square(y - labels))
    per_iter_err.append(avg_error)
    mse_derivative = tf.reduce_mean(2 * ( y - labels ))

    # update the weights and bias for the next epoch
    weight_update = tf.reduce_mean(mse_derivative*data)
    bias_update = tf.reduce_mean(mse_derivative)
    
    w = w - HP_lr * weight_update
    b = b - HP_lr * bias_update
    # end of iteration simulation

  final_epoch_error = np.array(per_iter_err).mean()
  err.append(final_epoch_error)
  #end of epoch simulation
  
  print('****')
  print('For epoch ',i+1,', the weights = ', w)


Phew! Observed how weights and bias correct themselves on each iteration, and each epoch? It's a long process!

Let's observe how the loss dropped over a period of time- on the y-axis we have the loss (mean squared error), and epochs on the x-axis

In [ ]:
import matplotlib.pyplot as plt
time_axis = range(1,len(err)+1)
plt.plot(  time_axis, err) 
plt.show()

After this, we can locate the point on the graph with minimum y (because this is an error). That's the point at which we can then retrain the model.

**Conclusion**

In this module we achieved the following:



*   Revised the model training process
*   Simulated the model training
*   Observed inbuilt TensorFlow functions
*   Initialized weights and bias
*   Observed their updates over iterations and epochs
*   Calculated and plotted the errors over epochs

